In [1]:
import os

In [4]:
%pwd

'/Users/shashwatkushwaha/Desktop/ML/TextSummarizer'

In [3]:
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [19]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [20]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-12-22 23:34:29,671 : INFO : common : yaml file : <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-12-22 23:34:29,755 : INFO : common : yaml file : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-12-22 23:34:29,756 : INFO : common : Created Directory at artifacts]
[2025-12-22 23:34:29,772 : INFO : common : Created Directory at artifacts/data_ingestion]
[2025-12-22 23:34:37,375 : INFO : 1805243024 : artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 8276685
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "479f594d7948681eacb4498edddd829150fb26eddbfe76d74275ed22477256ba"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 379C:2F7FB7:12674:1FCFA:6949883